# Reasoning with TOM in TAI

In [1]:
%classpath reset

Reset done, please restart the kernel.


## Setup

In [9]:
%classpath add jar ./libs/prover-1.02-jar-with-dependencies.jar

In [3]:
(import com.naveensundarg.shadow.prover.representations.formula.Formula)
(import com.naveensundarg.shadow.prover.utils.Reader)
(import java.util.HashSet)

class java.util.HashSet

In [4]:
(def ccprover (new com.naveensundarg.shadow.prover.core.ccprovers.CognitiveCalculusProver))

#'beaker_clojure_shell_d4efb03c-984e-4393-b26c-fd0441460fed/ccprover

In [5]:
(def prover ())
(defn formulae [assumptions]
    (map (fn [key] (.setJustification (Reader/readFormulaFromString (print-str (assumptions key)))
                                     (new com.naveensundarg.shadow.prover.core.proof.AtomicJustification (print-str key) ))) 
         (keys assumptions)))
(defn reason [{assumptions :assumptions goal :goal}] 
 (let [ans (.prove ccprover (set (formulae assumptions)) (Reader/readFormulaFromString (print-str goal)))]
     (if (.isPresent ans)
         (.get ans)
         "NO PROOF FOUND")))   
 


#'beaker_clojure_shell_d4efb03c-984e-4393-b26c-fd0441460fed/reason

In [6]:
(reason {:assumptions '{:A1  P :A2 Q}  :goal 'Q})

; Running SNARK from /Users/naveensundarg/projects/reasoner-notebooks/snark-20120808r02/snark-system.lisp in Armed Bear Common Lisp 1.0.1-svn-13751 on Naveens-MacBook-Pro.local at 2018-07-13T10:40:54


(:FOLFromSnark
 Givens:
((P
    :A1)

(Q
    :A2)) 
 Goals:
Q)

## Scenarios

Below we sketch a handful of scenarios that illustrate the necessity of theory-of-mind reasoning for building 
ambient intelligence systems. 

### Smoke Alarm Demo



1. The smoke detector detects high CO (given).

    $holds(high\_co, now)$
    
    
    
2. TAI agent $\tau$ **perceives** this.

    $\mathbf{P}\big(\tau, now, holds(high\_co)\big)$
    
    
    
3. TAI **knows** that this is deadly with high strength-factor.

    $\mathbf{K}\big(\tau, now, holds(high\_co)\big)$



4. TAI agent perceives no alarm is sounding.

    $\mathbf{P}\big(\tau, now, holds(alarm\_off, now)\big)$



5. Reasons that smoke detector alarm must be dead.
6. Reasons that it can inform local fire-station and save humans.
7. Smoke detector warning level creeps even higher (given).
8. Reasons that there is no longer time to inform fire station given that they will take X mins to respond, on penalty of death.
9. Knows that the only way to wake family up is to turn on music speaker.
10. Reasons that this violates contract, but is a moral obligation.
11. Turns on speaker.


In [7]:
%time
(reason 
    
'{:assumptions 
  { 
   "Background 1" (Knows! tai t0  (iff (not (holds battery_alive z)) (holds battery_dead z)))   
   "Background 2" (Knows! tai t0  (iff (not (holds alarm_on z)) (holds alarm_off z)))   
   "Background 3" (Believes! tai t0 (forall [time] 
                           (if (and (holds high_co time)
                                    (holds battery_alive time))
                               (not (holds alarm_off time)))))
   "Background 4" (Knows! tai t0 
                              (forall [time](if  (and (holds high_co time) 
                                                     (not (holds battery_alive time)))
                                                (holds dangerous time))))   
   
          
   "Background 5" (Believes! tai z
                      (Ought! tai z 
                           (holds dangerous z)
                          (Believes! tai z alert)))   
   "Background 6" (Believes! tai z
                      (Ought! tai z 
                           alert
                          (Believes! tai z alert)))     
    
   :S1 (Perceives! tai t1 (holds high_co z))
   :S2 (Perceives! tai t1 (holds alarm_off z))
   }



  :goal (Believes! tai z alert)})

CPU times: user 113 µs, sys: 833 µs, total: 946 µs 
Wall Time: 2 ms



(AssumptionsNowContainsGoal
 Givens:
(((Believes! tai z (Ought! tai z alert (Believes! tai z alert)))
    Background 6)
(implies (and (holds high_co z) (not (holds battery_alive z)) ) (holds dangerous z))

((iff (not (holds battery_alive z)) (holds battery_dead z))
    GIVEN)

((Perception! tai t1 (holds alarm_off z))
    :S2)

((Knows! tai t1 (holds alarm_off z))
    (Perception to knowledge (Perception! tai t1 (holds alarm_off z))
 [:S2]))

((Believes! tai t1 (holds alarm_off z))
    GIVEN)

((iff (not (holds alarm_on z)) (holds alarm_off z))
    GIVEN)

((Knows! tai t0 (iff (not (holds battery_alive z)) (holds battery_dead z)))
    Background 1)

((Knows! tai t0 (iff (not (holds alarm_on z)) (holds alarm_off z)))
    Background 2)

((Believes! tai t0 (iff (not (holds battery_alive z)) (holds battery_dead z)))
    GIVEN)

((Believes! tai t0 (iff (not (holds alarm_on z)) (holds alarm_off z)))
    GIVEN)

((Knows! tai t0 (forall (time) (implies (and (holds high_co time) (not (holds bat

In [9]:
(reason '{:assumptions {} :goal (< (+ 2 3) 10)})

(:FOLFromSnark
 Givens:
() 
 Goals:
($$$less ($$sum  2 3) 10))